In [1]:
import os
import scipy.io
import numpy as np
import pandas as pd
from custom import utils
from collections import Counter

## Analyses overview. 
This notebook outlines the steps for a bayesian model that attempts to find genomic information (SNPs) that are relavent for the disease (in this case schizophrenia). It does so by finding SNPs that explain the variance in a set of structural MRI thickness (sMRI) measurements well. It then finds the sMRI features which predict the diagnosis well and. Together it uses the fact that G (SNPs) -> I (sMRI) -> Y (diagnosis) to prepose that the set of SNPs (G) which explain the variance well in the set of sMRI features (I) that are important for predicting diagnosis, are important genomic locations for the disease.

### Function for preprocessing
The <b>most_handed</b> function performs categorical imputation. This decision is made based on the fact that ~90 out of 1000+ handed values are missing. That small percentage is imputed with the most frequent handedness score "Right". There's also a very small percentage of non right non left values that are a mix of "Both", "Mixed", "Either", "Ambidextrous", these are all imputed to just "Both". The other function mean centers and scales the data. 

In [2]:
def most_handed(data):
    """data: pd.DataFrame"""
    counts = Counter(data)
    most = max(counts.items())[0]
    data = data.copy().fillna(0)
    data[data == 0] = 'Right'
    both = ['Both','Mixed','Either','Ambidextrous']
    for hand in both:
        data[data == hand] = 'Both'
    return data

def mean_center_n_scale(data):
    """data: np.ndarray"""
    data_mean_zero = data - data.mean(0)
    data_scaled = data_mean_zero / data_mean_zero.std(0)
    return data_scaled

## Preprocessing function
Inputs are paths to data, sMRI features and SNPs respectively. "bcvar" is a list of covariates: ['SEX', 'AGE_MRI', 'EstimatedTotalIntraCranialVol', 'STUDY']. "brain_cols" is a numpy array of feature names to subset the sMRI data. 
First thing that happens is I get a boolen array of where the brain data contains only Controls or Schizophrenics in the Group feature. Both datasets are subsetted by this boolean array, row wise. It's important to note that the datasets loaded are already in the same order rowwise. Then I create 2 sets of covariate matrices and concatenate them into one. The first set contains AGE, SEX, and ICV(EstimatedTotalIntraCranialVol), the second one is a one hot encoded matrix of handedness, the third is a one hot encoded matrix of site(i.e, study). The first step of the analysis does not use the response variable but I safe it to apply stratifiedKFold cross-validation. Two dictionaries are returned, one contains keys and numpy arrays. The name of the keys are required inputs for the first part of the analysis. The dictionary containing the column headers is not required but is saved for later uses. This function also applies mean centering and scaling

In [3]:
def preprocess(brain_path, snp_path, bcvar, brain_cols):
    """brain_path: String, snp_path: String, bcvar: list, 
    brain_cols: list or np.ndarray. 
    """
    # load data
    brain_data = pd.read_hdf(brain_path)
    snp_data = pd.read_hdf(snp_path)
    # get the group status
    gr = brain_data.GROUP.values
    cnt_scz = np.logical_or(gr == 'Control', gr == 'Schizophrenia')
    # subset by indexes cnt_scz
    brain_data = brain_data.iloc[cnt_scz, :]
    snp_data = snp_data.iloc[cnt_scz, :]
    # create set of covariates
    icv = 'EstimatedTotalIntraCranialVol'
    cov_set1 = pd.DataFrame(
        data=np.hstack((snp_data.SEX.values[:, None],
                        brain_data.AGE_MRI.values[:, None],
                        brain_data[icv].values[:, None])),
        columns=['SEX','AGE','EstimatedTotalIntraCranialVol'])
    cov_set1 = cov_set1.fillna(0)
    cov_set1[cov_set1.AGE == 0] = cov_set1.AGE.mean()
    cov_site = utils.make_non_singular(utils.encoder(brain_data.STUDY.values))
    cov_site_cols = ['site{}'.format(i) for i in range(cov_site.shape[1])]
    cov_site = pd.DataFrame(data=cov_site, columns=cov_site_cols)
    cov_hand = utils.encoder(most_handed(brain_data.HANDED))
    cov_hand_cols = ['handed{}'.format(i) for i in range(cov_hand.shape[1])]
    cov_hand = pd.DataFrame(data=cov_hand, columns=cov_hand_cols)
    cvars = pd.concat([cov_set1, cov_site, cov_hand], axis=1)
    cvars_val = utils.make_non_singular(mean_center_n_scale(cvars.values))
    y = np.array([0 if i == 'Control' else 1 for i in brain_data.GROUP.values])
    return {'Z': cvars_val, 
            'I': mean_center_n_scale(brain_data[brain_cols].values), 
            'G': mean_center_n_scale(snp_data.iloc[:, 1:-5].values),
            'colnames': snp_data.iloc[:, 1:-5].columns.values,
            'y':y}, {'Z_cols': cvars.columns.values,
                     'I_cols': brain_cols,
                     'G_cols': snp_data.iloc[:, 1:-5].columns.values}

# Helper functions
These two functions assist in the analysis. save_preprocessed saves the data to disk that are outputed from the preprocess function above. The paths to where the files are written on disk are returned. This is because I'll be using nipype so to make life easier data isn't passed when interfacing with nipype nodes - just the path to where the data lives. I then load the data using the paths. The cv_maker function creates k-fold stratified cross validation indices and saves them. These indices are used in the matlab script to load the correct subsets of data. 

In [5]:
def save_preprocessed(preproc_data_dict, preproc_data_dict_col, save_path, dn, cn):
    """preproc_data_dict: dictionary object returned from
    the preprocessing function, (the first - zeroth value of the return) 
    preproc_data_dict_col: dintionary object returned from
    the preprocessing function, (the second - first value of the return)
    save_path: string - base path for saving the dictionaries
    dn: string - name for saving the data dictionary 
    cn: string - name for saving the column header dictionary
    """
    save_dict = os.path.join(save_path, dn)
    save_cols = os.path.join(save_path, cn)
    scipy.io.savemat(save_dict, mdict=preproc_data_dict)
    utils.save_pickle(save_cols, preproc_data_dict_col)
    return save_dict, save_cols

def cv_maker(data_path, save_path):
    """data_path: string
    save_path: string
    """
    import scipy.io
    from sklearn.model_selection import StratifiedKFold
    X = scipy.io.loadmat(data_path)['I']
    y = scipy.io.loadmat(data_path)['y'][0]
    cv = StratifiedKFold(n_splits=5, random_state=1)
    train_idx, test_idx = {}, {}
    for idx, (train, test) in enumerate(cv.split(X, y)):
        train_idx['train_{}'.format(idx + 1)] = train + 1
        test_idx['test_{}'.format(idx + 1)] = test + 1
    scipy.io.savemat(save_path, mdict={"train":train_idx, "test":test_idx})
    return save_path

# Create the input data, save the CV indices
The step below runs the functions I've made above

In [14]:
headers_dir = "/storage/gablab001/data/genus/GIT/genus/fs_cog/pred_diag/column_headers"
brain_cols = np.genfromtxt(os.path.join(headers_dir, "XB"), dtype=str)
brain_path = "/storage/gablab001/data/genus/GIT/genus/bayes/data_sets/brain_N1547_P5927_matched.hdf5py"
snp_path = "/storage/gablab001/data/genus/GIT/genus/bayes/data_sets/genomic_N1547_P100006_matched.hdf5py"
bcv = ['SEX', 'AGE_MRI', 'EstimatedTotalIntraCranialVol', 'STUDY']
all_data, all_cols = preprocess(brain_path, snp_path, bcv, brain_cols)
path_for_save = "/storage/gablab001/data/genus/GIT/genus/bayes/data_sets"
#for_cv, _ = save_preprocessed(all_data, all_cols, path_for_save, "brain_gene.mat","brain_gene_cols.pkl")
#cv_path = cv_maker(for_cv, os.path.join(path_for_save, "cv_idx.mat"))

In [2]:
from nipype import Function, Node, Workflow, IdentityInterface

# Bayesian analysis - "step 1" 
This step attempts to find a subset of SNPs per each feature in the sMRI data that explain the variance well in that feature. Below I create the workflow that I use with nipype, create the nipype wrapper nodes to wrap functions that will go into the nipype graph, and then submit the jobs. Due to the nature of the analysis we are parallelizing over the feature space in the sMRI data. That is - one job per feature, on top of that we are parallelizing the cross validation step. In total this means there are (170*10) jobs that need to be submitted. For a single user in my experience that's too many jobs for the Openmind cluster so I limit the amount of jobs that can be submitted at a time. 

In [7]:
wf_bf = Workflow(name='brain_bcv')
wf_bf.base_dir = "/om/scratch/Tue/ysa"

Iternode = Node(IdentityInterface(fields=['col_idx', 'cv_idx']), name = 'Iternode')
Iternode.iterables = [('col_idx', np.arange(170) + 1), ('cv_idx', np.arange(5) + 1)]

def run_bayes(in_file, cv_file, cv_idx, col_idx, out_file):
    """in_file: string - path to base data file
    cv_file: string - path to cross validation indices stored in a .mat file
    cv_idx: int - ranges from 1 to K+1, K = number of CV folds
    col_idx: int - ranges from 1 to P+1, P = number of columns in sMRI features
    out_file: string - path to where the output is saved
    """
    import os
    import numpy as np
    import cPickle as pickle
    import nipype.interfaces.matlab as Matlab
    def outnames(col, out):
        return os.path.join(out, '{}.mat'.format(col))
    headers_dir = "/storage/gablab001/data/genus/GIT/genus/fs_cog/pred_diag/column_headers"
    col_names = np.genfromtxt(os.path.join(headers_dir, "XB"), dtype=str)
    col_save_name = col_names[col_idx - 1] + "_{}_{}_BF".format(cv_idx, col_idx)
    with open("/storage/gablab001/data/genus/GIT/genus/bayes/matlab/bayes_reg.m", "r") as src:
        script = src.read().replace("\n", "")
    mat_file = outnames(in_file[:-4]+'_'+col_save_name, out_file)
    matlab = Matlab.MatlabCommand()
    matlab.inputs.script = script.format(in_file, cv_file, cv_idx, col_idx, mat_file)
    res = matlab.run()
    return mat_file

Run_bayes = Node(interface=Function(
    input_names = ['in_file', 'cv_file','cv_idx',
                   'col_idx','out_file'],
    output_names = ['mat_file'],
    function = run_bayes
), name='Run_bayes')

Run_bayes.inputs.in_file = "/storage/gablab001/data/genus/GIT/genus/bayes/data_sets/brain_gene.mat"
Run_bayes.inputs.cv_file = "/storage/gablab001/data/genus/GIT/genus/bayes/data_sets/cv_idx.mat"
Run_bayes.inputs.out_file = "/storage/gablab001/data/genus/GIT/genus/bayes/results/bayes_factor"
wf_bf.connect(Iternode, 'cv_idx', Run_bayes, 'cv_idx')
wf_bf.connect(Iternode, 'col_idx', Run_bayes, 'col_idx')
#wf.run(plugin='SLURM', plugin_args={'sbatch_args':'--mem=4G -t 23:00:00', 'max_jobs': 170})

# Set up for step 2
Step 1 above uses an updated version of varbvs and in <b>bayes_reg.m</b> the normalization step is already performed. The matlab file <b>deployendophenVB.m</b> wants a csv file for step 2 that points to the output from step 1. The functions below create one of those for each fold. So 5 folds, 5 files where each file has 170 paths. 

In [134]:
bayes_factor_outputs = "/storage/gablab001/data/genus/GIT/genus/bayes/results/bayes_factor/5foldcv_allcov"
bf_res = [os.path.join(bayes_factor_outputs,x) for x in os.listdir(bayes_factor_outputs)]

def make_file(paths):
    cv = [i.split('_')[-3] for i in paths]
    df = pd.DataFrame({'path':paths, 'cv':cv})
    groups = df.groupby('cv')
    df_store = []
    for group in groups.groups:
        gr = groups.get_group(group).reset_index(drop=True).drop('cv', 1)
        gr['sorter'] = [int(i.split('_')[-2]) for i in gr.path]
        gr = gr.sort_values('sorter').reset_index(drop=True)
        gr.columns=['matFn','colNum']
        df_store.append(gr[['colNum', 'matFn']])
    return df_store
 
#for idx, f in enumerate(make_file(bf_res)):
#    f.to_csv('BFRESULT_CV_{}.csv'.format(idx+1), index=None)

# Fixed form variational bayes "Step 2"


In [ ]:
wf_fxvb = Workflow(name='fxvb_step')
wf_fxvb.base_dir = '/om/scratch/Fri/ysa'

def run_fxvb(in_file, csv_file, out_file):
    """in_file: string - path to base input data file
    csv_file: string - path to a template of the csv_file created 
              by "make_file" func
    out_file: string - path for the output of run_fxvb function
    """
    import os
    import numpy as np
    import nipype.interfaces.matlab as Matlab
    with open("/storage/gablab001/data/genus/GIT/genus/bayes/matlab/fxvb.m", "r") as src:
        script = src.read().replace("\n", "")
    mat_name = 'fxvb_'+csv_file.split('/')[-1][-8:-4]+'.mat'
    mat_file = os.path.join(out_file, mat_name)
    matlab = Matlab.MatlabCommand()
    matlab.inputs.script = script.format(csv_file, in_file, mat_file)
    print(script.format(csv_file, in_file, mat_file))
    res = matlab.run()  
    return mat_file
    
Run_fxvb = Node(interface=Function(
    input_names = ['in_file', 'csv_file','out_file'],
    output_names = ['mat_file'],
    function = run_fxvb
), name='Run_fxvb')

cvpath = "/storage/gablab001/data/genus/GIT/genus/bayes/results/bayes_factor"
IterCV = Node(IdentityInterface(fields=['csv_file']), name = 'Iternode')
IterCV.iterables = ('csv_file', [os.path.join(cvpath, x) for x in os.listdir(cvpath) if "_CV_" in x])
Run_fxvb.inputs.in_file = '/storage/gablab001/data/genus/GIT/genus/bayes/data_sets/brain_gene.mat'
Run_fxvb.inputs.out_file = '/storage/gablab001/data/genus/GIT/genus/bayes/results/fxvb'
wf_fxvb.connect(IterCV, 'csv_file', Run_fxvb, 'csv_file')
wf_fxvb.run(plugin='SLURM', plugin_args={'sbatch_args':'--mem=10G -t 4-23:00:00'})

170728-11:45:31,213 workflow INFO:
	 Workflow fxvb_step settings: ['check', 'execution', 'logging']
170728-11:45:31,235 workflow INFO:
	 Running in parallel.
170728-11:45:31,239 workflow INFO:
	 Pending[0] Submitting[5] jobs Slots[20]
170728-11:45:31,240 workflow INFO:
	 Submitting: Run_fxvb.a3 ID: 0
170728-11:45:31,311 workflow INFO:
	 Finished submitting: Run_fxvb.a3 ID: 0
170728-11:45:31,312 workflow INFO:
	 Submitting: Run_fxvb.a2 ID: 1
170728-11:45:31,381 workflow INFO:
	 Finished submitting: Run_fxvb.a2 ID: 1
170728-11:45:31,382 workflow INFO:
	 Submitting: Run_fxvb.a4 ID: 2
170728-11:45:31,451 workflow INFO:
	 Finished submitting: Run_fxvb.a4 ID: 2
170728-11:45:31,453 workflow INFO:
	 Submitting: Run_fxvb.a1 ID: 3
170728-11:45:31,519 workflow INFO:
	 Finished submitting: Run_fxvb.a1 ID: 3
170728-11:45:31,521 workflow INFO:
	 Submitting: Run_fxvb.a0 ID: 4
170728-11:45:31,585 workflow INFO:
	 Finished submitting: Run_fxvb.a0 ID: 4


### Single runs because slurm is just weird

In [3]:
import nipype.interfaces.matlab as Matlab

In [6]:
with open("/storage/gablab001/data/genus/GIT/genus/bayes/matlab/fxvb.m", "r") as src:
        script = src.read().replace("\n", "")
csv_file = "/storage/gablab001/data/genus/GIT/genus/bayes/results/bayes_factor/BFRESULT_CV_1.csv"
in_file = "/storage/gablab001/data/genus/GIT/genus/bayes/data_sets/brain_gene.mat"
out_file = "/storage/gablab001/data/genus/GIT/genus/bayes/results/fxvb"
mat_name = 'fxvb_'+csv_file.split('/')[-1][-8:-4]+'.mat'
mat_file = os.path.join(out_file, mat_name)
matlab = Matlab.MatlabCommand()
matlab.inputs.script = script.format(csv_file, in_file, mat_file)
res = matlab.run()